<a href="https://colab.research.google.com/github/rediahmds/eco-sort/blob/main/train/build_dataset_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [1]:
USE_COLAB = False

if USE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
else:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive

    try:
        !pip install PyDrive2

        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("GCP_CLIENT_SECRET")

        with open("client_secrets.json", "w") as f:
            f.write(secret_value_0)


        gauth = GoogleAuth()
        gauth.CommandLineAuth()

        drive = GoogleDrive(gauth)
    except FileNotFoundError as fne:
        print("Could not find credentials from GCP named client_secrets.json")
        print("Error:", fne)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 71.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 44.0.3
    Uninstalling cryptography-44.0.3:
      Successfully uninstalled cryptography-44.0.3
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 25.1.0
    Uninstalling pyOpenSSL-25.1.0:
      Successfully uninstalled pyOpenSSL-25.1.0
Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=991309652978-hpmqspbdt0bccglf6iklc3er1d9naa8q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code



Enter verification code:  4/1AVMBsJhzil5kUEbJeBJS9YoK74XW8fRtHMK56GW-6LG2CrYLl04MFNcsp_U


Authentication successful.


## Prepare Dataset

### Download Dataset from Kaggle

The dataset comes from different accounts on Kaggle.

In [ ]:
!pip install kagglehub

In [2]:
import kagglehub

# alistair_ds = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")
# print("Path to dataset files:", alistair_ds)

# mostafa_ds = kagglehub.dataset_download("mostafaabla/garbage-classification")
# print("Path to dataset files:", mostafa_ds)

joe_ds = kagglehub.dataset_download("joebeachcapital/realwaste")
print("Path to dataset files:", joe_ds)

# glhdamar_ds = kagglehub.dataset_download("glhdamar/new-trash-classfication-dataset")
# print("Path to dataset files:", glhdamar_ds)

Path to dataset files: /kaggle/input/realwaste


### Show Directory Tree

In [3]:
from pathlib import Path

def print_directory_tree(root: Path, prefix: str = ""):
    """
    Mencetak struktur direktori dengan tampilan seperti pohon.
    Hanya menampilkan folder (tanpa file).
    """
    subdirs = sorted([p for p in root.iterdir() if p.is_dir()])
    for i, subdir in enumerate(subdirs):
        connector = "└── " if i == len(subdirs) - 1 else "├── "
        print(f"{prefix}{connector}{subdir.name}")
        extension = "    " if i == len(subdirs) - 1 else "│   "
        print_directory_tree(subdir, prefix + extension)

In [4]:
# Path ke folder utama
# alistair_path = Path(alistair_ds) / "images" / "images"
# mostafa_path = Path(mostafa_ds) / "garbage_classification"
joe_path = Path(joe_ds) / "realwaste-main" / "RealWaste"
# glhdamar_path = Path(glhdamar_ds) / "new-dataset-trash-type-v2"

# print(alistair_path.name)
# print_directory_tree(alistair_path)

# print(mostafa_path.name)
# print_directory_tree(mostafa_path)

print(joe_path.name)
print_directory_tree(joe_path)

# print(glhdamar_path.name)
# print_directory_tree(glhdamar_path)

RealWaste
├── Cardboard
├── Food Organics
├── Glass
├── Metal
├── Miscellaneous Trash
├── Paper
├── Plastic
├── Textile Trash
└── Vegetation


### Download Existing Dataset from Google Drive

In [5]:
if USE_COLAB:
    !cp /content/drive/MyDrive/Cool Lee Yeah/8th Semester/Skripsi/Datasets/dataset.7z .
else:
    from kaggle_secrets import UserSecretsClient
    GDRIVE_DATASET_FILE_ID = user_secrets.get_secret("GDRIVE_DATASET_FILE_ID")
    file_id = GDRIVE_DATASET_FILE_ID
    local_filename = "dataset.7z" # save as

    try:
        file_to_download = drive.CreateFile({'id': file_id})

        print(f"⬇️ Downloading file: '{file_to_download['title']}'...")
        file_to_download.GetContentFile(local_filename)

        print(f"✅ Downloaded successfully and saved as '{local_filename}'")

    except Exception as e:
        print(f"Error: {e}")


⬇️ Downloading file: 'dataset.7z'...
✅ Downloaded successfully and saved as 'dataset.7z'


In [6]:
!7z x dataset.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 850848634 bytes (812 MiB)

Extracting archive: dataset.7z
--
Path = dataset.7z
Type = 7z
Physical Size = 850848634
Headers Size = 152784
Method = LZMA2:24
Solid = +
Blocks = 1

      0%      0% 607 - dataset/train/glass/default_gl . rage_bottles_0004__Image_37.pn                                                                            1% 689 - dataset/train/glass/default_gl . rage_bottles_0125__Image_29.pn                                                                            1% 780 - dataset/train/glass/default_glas . c_containers_0031__Image_243.p                                                                              2% 858 - dataset/train/glass/default_glas . c_containers_0164__Image_216.p                                

In [7]:
print_directory_tree(Path("dataset/train"))

├── glass
├── metal
├── organic
├── paper
├── plastic
├── styrofoam
└── textiles


### Merge All Dataset

Copy from different sources.

In [8]:
from pathlib import Path
import shutil
import random

def copy_n_files(src_dir, dst_dir, n, randomize=False):
    src_path = Path(src_dir)
    dst_path = Path(dst_dir)

    # Buat folder tujuan jika belum ada
    dst_path.mkdir(parents=True, exist_ok=True)

    # Ambil semua file dari direktori sumber
    all_files = [f for f in src_path.iterdir() if f.is_file()]

    # Pastikan n tidak lebih besar dari jumlah file
    n = min(n, len(all_files))

    # Tentukan file mana yang akan disalin
    if randomize:
        files_to_copy = random.sample(all_files, n)
    else:
        files_to_copy = sorted(all_files)[:n]

    # Copy file satu per satu
    for file in files_to_copy:
        shutil.copy(file, dst_path)
        print(f"Copied: {file.name}")

    print(f"\nTotal {n} files copied from '{src_dir}' to '{dst_dir}' (random: {randomize}).")

In [22]:
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Plastic", "dataset/train/plastic", 920, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Paper", "dataset/train/paper", 500, randomize=True)
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Cardboard", "dataset/train/paper", 461, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Food Organics", "dataset/train/organic", 411, randomize=True)
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Vegetation", "dataset/train/organic", 436, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Glass", "dataset/train/glass", 420, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Metal", "dataset/train/metal", 420, randomize=True)






Copied: Glass_144.jpg
Copied: Glass_258.jpg
Copied: Glass_315.jpg
Copied: Glass_35.jpg
Copied: Glass_392.jpg
Copied: Glass_393.jpg
Copied: Glass_113.jpg
Copied: Glass_275.jpg
Copied: Glass_228.jpg
Copied: Glass_207.jpg
Copied: Glass_359.jpg
Copied: Glass_31.jpg
Copied: Glass_249.jpg
Copied: Glass_231.jpg
Copied: Glass_64.jpg
Copied: Glass_77.jpg
Copied: Glass_199.jpg
Copied: Glass_132.jpg
Copied: Glass_106.jpg
Copied: Glass_389.jpg
Copied: Glass_217.jpg
Copied: Glass_185.jpg
Copied: Glass_193.jpg
Copied: Glass_7.jpg
Copied: Glass_289.jpg
Copied: Glass_204.jpg
Copied: Glass_180.jpg
Copied: Glass_416.jpg
Copied: Glass_354.jpg
Copied: Glass_227.jpg
Copied: Glass_253.jpg
Copied: Glass_379.jpg
Copied: Glass_161.jpg
Copied: Glass_274.jpg
Copied: Glass_251.jpg
Copied: Glass_320.jpg
Copied: Glass_58.jpg
Copied: Glass_78.jpg
Copied: Glass_294.jpg
Copied: Glass_114.jpg
Copied: Glass_313.jpg
Copied: Glass_135.jpg
Copied: Glass_300.jpg
Copied: Glass_277.jpg
Copied: Glass_335.jpg
Copied: Glass_310.

### Check for Duplicates

#### Helper: Auto Reconnect

In [ ]:
#@title Time Out Preventer (Advanced)
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [10]:
!pip install imagehash

In [11]:
from pathlib import Path
from PIL import Image
import imagehash
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from collections import defaultdict
import matplotlib.pyplot as plt

def get_resnet_embedding(img: Image.Image, model, transform, device):
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(img_tensor).squeeze()
    return features.cpu()


def find_duplicates_images_with_end_deletion(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    # === Inisialisasi hash & CNN
    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            # 🔁 Bandingkan dengan semua hash yang mirip
            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                duplicates.append(img_path)

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                print("\n")

                                break  # cukup validasi 1 yang cocok

                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # keluar dari hash loop jika sudah match

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    # === Hapus jika diminta
    if delete:
        for dup in duplicates:
            try:
                dup.unlink()
                print(f"🗑️ Menghapus: {dup}")
            except Exception as e:
                print(f"❌ Gagal menghapus {dup}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {len(duplicates)}")
    return duplicates


def find_duplicates_images_with_immediate_deletion(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                if delete:
                                    try:
                                        img_path.unlink()
                                        print(f"🗑️ Duplikat dihapus: {img_path}")
                                    except Exception as e:
                                        print(f"❌ Gagal menghapus {img_path}: {e}")
                                else:
                                    duplicates.append(img_path)

                                print("\n")
                                break  # match pertama cukup
                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # hash cocok, stop

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {counter}")
    return duplicates

In [21]:
duplicates = find_duplicates_images_with_end_deletion(
    "dataset/train/glass",
    hash_threshold=6,
    delete=True,
    show_preview=False,
)

🔍 Mendeteksi duplikat di dalam: /kaggle/working/dataset/train/organic
🔢 Kode duplikat: 1
⚠️ Duplikat terdeteksi:
  Original : Food Organics_81.jpg
  Duplicate: Food Organics_82.jpg
  🔗 Cosine Similarity: 0.9886


🗑️ Menghapus: dataset/train/organic/Food Organics_82.jpg

✅ Total duplikat terverifikasi: 1


### Check Dataset Distribution

In [23]:
from collections import Counter
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("dataset/train")
label_counts = Counter([label for _, label in train_dataset])
print("Label mapping:", train_dataset.class_to_idx)
print("Distribusi kelas:", label_counts)

Label mapping: {'glass': 0, 'metal': 1, 'organic': 2, 'paper': 3, 'plastic': 4, 'styrofoam': 5, 'textiles': 6}
Distribusi kelas: Counter({3: 4042, 6: 3892, 4: 3264, 2: 3210, 0: 3109, 1: 2653, 5: 521})


## Archive Dataset and Save to Google Drive

In [24]:
!rm dataset.7z
!7z a dataset.7z dataset/

if USE_COLAB:
    !cp dataset.7z /content/drive/MyDrive/Cool Lee Yeah/8th Semester/Skripsi/Datasets/
else:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GDRIVE_DATASET_FOLDER_ID = user_secrets.get_secret("GDRIVE_DATASET_FOLDER_ID")

    destination_folder_id = GDRIVE_DATASET_FOLDER_ID
    upload_target_file = "dataset.7z"

    file_to_upload = drive.CreateFile(
        {
            "title": upload_target_file,
            "parents": [{"id": destination_folder_id}]
        }
    )

    print(f"⬆️ Uploading {file_to_upload['title']}...")
    file_to_upload.SetContentFile(upload_target_file)
    file_to_upload.Upload()
    print(f"✅ Uploaded successfully")



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive:
  0M Sca        9 folders, 20691 files, 1261537781 bytes (1204 MiB)

Creating archive: dataset.7z

Items to compress: 20700

      0% 2793 + dataset/train/glass/white-glass428.j                                                  0% 3277 + dataset/train/metal/Metal_259.jp                                              1% 3277 + dataset/train/metal/Metal_259.jp                                              2% 3277 + dataset/train/metal/Metal_259.jp                                              3% 3277 + dataset/train/metal/Metal_259.jp                                              4% 3277 + dataset/train/metal/Metal_259.jp                                              5% 3277 + dataset/train/metal/Metal_259.jp                                              6% 3277 + dataset/train/met